In [390]:
# importing basic libraries
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score,r2_score
from sklearn.ensemble import RandomForestClassifier

### Data Gathering

In [293]:
deliveries=pd.read_csv(r"C:\Users\pooja\OneDrive\Desktop\ipl-rough\deliveries.csv")
deliveries.sample(7)

,match_id,inning,batting_team,bowling_team,over,ball,batter,bowler,non_striker,batsman_runs,extra_runs,total_runs,extras_type,is_wicket,player_dismissed,dismissal_kind,fielder
178597,1181767,1,Delhi Capitals,Chennai Super Kings,4,1,S Dhawan,DL Chahar,C Munro,1,0,1,NaN,0,NaN,NaN,NaN
256528,1426291,1,Chennai Super Kings,Punjab Kings,10,1,MM Ali,HV Patel,RA Jadeja,0,0,0,NaN,0,NaN,NaN,NaN
243500,1370352,2,Mumbai Indians,Gujarat Titans,2,2,RG Sharma,Mohammed Shami,SA Yadav,0,0,0,NaN,1,RG Sharma,caught,J Little
9506,336023,1,Rajasthan Royals,Royal Challengers Bangalore,11,2,GC Smith,V Kohli,SA Asnodkar,1,0,1,NaN,0,NaN,NaN,NaN
163362,1136615,1,Delhi Daredevils,Mumbai Indians,14,5,RR Pant,Mustafizur Rahman,V Shankar,1,0,1,NaN,0,NaN,NaN,NaN
222101,1304105,1,Chennai Super Kings,Mumbai Indians,3,4,AT Rayudu,JJ Bumrah,RD Gaikwad,4,0,4,NaN,0,NaN,NaN,NaN
15029,392189,1,Delhi Daredevils,Chennai Super Kings,18,6,AB de Villiers,A Flintoff,MK Tiwary,1,0,1,NaN,0,NaN,NaN,NaN


In [294]:
matches=pd.read_csv(r"C:\Users\pooja\OneDrive\Desktop\ipl-rough\matches.csv")
matches.sample(7)

,id,season,city,date,match_type,player_of_match,venue,team1,team2,toss_winner,toss_decision,winner,result,result_margin,target_runs,target_overs,super_over,method,umpire1,umpire2
114,392239,2009,Johannesburg,2009-05-24,Final,A Kumble,New Wanderers Stadium,Royal Challengers Bangalore,Deccan Chargers,Royal Challengers Bangalore,field,Deccan Chargers,runs,6.0,144.0,20.0,N,NaN,RE Koertzen,SJA Taufel
855,1254100,2021,Dubai,2021-09-27,League,JJ Roy,Dubai International Cricket Stadium,Rajasthan Royals,Sunrisers Hyderabad,Rajasthan Royals,bat,Sunrisers Hyderabad,wickets,7.0,165.0,20.0,N,NaN,KN Ananthapadmanabhan,Navdeep Singh
870,1254088,2021,Abu Dhabi,2021-10-08,League,Ishan Kishan,"Zayed Cricket Stadium, Abu Dhabi",Mumbai Indians,Sunrisers Hyderabad,Mumbai Indians,bat,Mumbai Indians,runs,42.0,236.0,20.0,N,NaN,Tapan Sharma,VK Sharma
529,980925,2016,Chandigarh,2016-04-19,League,RV Uthappa,"Punjab Cricket Association IS Bindra Stadium, ...",Kings XI Punjab,Kolkata Knight Riders,Kolkata Knight Riders,field,Kolkata Knight Riders,wickets,6.0,139.0,20.0,N,NaN,S Ravi,C Shamshuddin
545,980957,2016,Pune,2016-05-01,League,RG Sharma,Maharashtra Cricket Association Stadium,Rising Pune Supergiants,Mumbai Indians,Mumbai Indians,field,Mumbai Indians,wickets,8.0,160.0,20.0,N,NaN,AY Dandekar,RJ Tucker
975,1359500,2023,Jaipur,2023-04-19,League,MP Stoinis,"Sawai Mansingh Stadium, Jaipur",Lucknow Super Giants,Rajasthan Royals,Rajasthan Royals,field,Lucknow Super Giants,runs,10.0,155.0,20.0,N,NaN,J Madanagopal,YC Barde
501,829793,2015,Raipur,2015-05-09,League,MC Henriques,Shaheed Veer Narayan Singh International Stadium,Delhi Daredevils,Sunrisers Hyderabad,Sunrisers Hyderabad,bat,Sunrisers Hyderabad,runs,6.0,164.0,20.0,N,NaN,VA Kulkarni,S Ravi


### Data Cleaning

In [296]:
deliveries.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 260920 entries, 0 to 260919
Data columns (total 17 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   match_id          260920 non-null  int64 
 1   inning            260920 non-null  int64 
 2   batting_team      260920 non-null  object
 3   bowling_team      260920 non-null  object
 4   over              260920 non-null  int64 
 5   ball              260920 non-null  int64 
 6   batter            260920 non-null  object
 7   bowler            260920 non-null  object
 8   non_striker       260920 non-null  object
 9   batsman_runs      260920 non-null  int64 
 10  extra_runs        260920 non-null  int64 
 11  total_runs        260920 non-null  int64 
 12  extras_type       14125 non-null   object
 13  is_wicket         260920 non-null  int64 
 14  player_dismissed  12950 non-null   object
 15  dismissal_kind    12950 non-null   object
 16  fielder           9354 non-null    obj

In [297]:
deliveries['batting_team'].unique()

array(['Kolkata Knight Riders', 'Royal Challengers Bangalore',
       'Chennai Super Kings', 'Kings XI Punjab', 'Rajasthan Royals',
       'Delhi Daredevils', 'Mumbai Indians', 'Deccan Chargers',
       'Kochi Tuskers Kerala', 'Pune Warriors', 'Sunrisers Hyderabad',
       'Rising Pune Supergiants', 'Gujarat Lions',
       'Rising Pune Supergiant', 'Delhi Capitals', 'Punjab Kings',
       'Lucknow Super Giants', 'Gujarat Titans',
       'Royal Challengers Bengaluru'], dtype=object)

1. due to change in names of teams their are many teams so will convert into 8 teams.
2. as we are creating predictor we will consider only present 8 teams.

In [299]:
def change_name(team_name):
    if team_name in ['Rising Pune Supergiants','Pune Warriors']:
        return 'Rising Pune Supergiant'
    elif team_name=='Royal Challengers Bengaluru':
        return 'Royal Challengers Bangalore'
    elif team_name=='Deccan Chargers':
        return 'Sunrisers Hyderabad'
    elif team_name == 'Delhi Daredevils':
        return 'Delhi Capitals'
    elif team_name== 'Gujarat Lions':
        return 'Gujarat Titans'
    elif team_name == 'Kings XI Punjab':
        return 'Punjab Kings'
    else:
        return team_name

In [300]:
deliveries['batting_team']=deliveries['batting_team'].apply(change_name)
deliveries['bowling_team']=deliveries['bowling_team'].apply(change_name)
matches['team1']=matches['team1'].apply(change_name)
matches['team2']=matches['team2'].apply(change_name)

In [301]:
deliveries=deliveries[(deliveries['batting_team']!='Kochi Tuskers Kerala')&(deliveries['bowling_team']!='Kochi Tuskers Kerala')]
deliveries=deliveries[(deliveries['batting_team']!='Rising Pune Supergiant')&(deliveries['bowling_team']!='Rising Pune Supergiant')]
matches=matches[(matches['team1']!='Kochi Tuskers Kerala')&(matches['team2']!='Kochi Tuskers Kerala')]
matches=matches[(matches['team1']!='Rising Pune Supergiant')&(matches['team2']!='Rising Pune Supergiant')]

In [302]:
deliveries.info()

<class 'pandas.core.frame.DataFrame'>
Index: 240047 entries, 0 to 260919
Data columns (total 17 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   match_id          240047 non-null  int64 
 1   inning            240047 non-null  int64 
 2   batting_team      240047 non-null  object
 3   bowling_team      240047 non-null  object
 4   over              240047 non-null  int64 
 5   ball              240047 non-null  int64 
 6   batter            240047 non-null  object
 7   bowler            240047 non-null  object
 8   non_striker       240047 non-null  object
 9   batsman_runs      240047 non-null  int64 
 10  extra_runs        240047 non-null  int64 
 11  total_runs        240047 non-null  int64 
 12  extras_type       13091 non-null   object
 13  is_wicket         240047 non-null  int64 
 14  player_dismissed  11921 non-null   object
 15  dismissal_kind    11921 non-null   object
 16  fielder           8628 non-null    object
d

In [303]:
matches.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1006 entries, 0 to 1094
Data columns (total 20 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   id               1006 non-null   int64  
 1   season           1006 non-null   object 
 2   city             955 non-null    object 
 3   date             1006 non-null   object 
 4   match_type       1006 non-null   object 
 5   player_of_match  1002 non-null   object 
 6   venue            1006 non-null   object 
 7   team1            1006 non-null   object 
 8   team2            1006 non-null   object 
 9   toss_winner      1006 non-null   object 
 10  toss_decision    1006 non-null   object 
 11  winner           1002 non-null   object 
 12  result           1006 non-null   object 
 13  result_margin    988 non-null    float64
 14  target_runs      1004 non-null   float64
 15  target_overs     1004 non-null   float64
 16  super_over       1006 non-null   object 
 17  method           17

## runs left

In [305]:
total_runs_df=deliveries.groupby(['match_id','inning'])['total_runs'].sum().reset_index()
total_runs_df

,match_id,inning,total_runs
0,335982,1,222
1,335982,2,82
2,335983,1,240
3,335983,2,207
4,335984,1,129
...,...,...,...
2035,1426310,2,174
2036,1426311,1,175
2037,1426311,2,139
2038,1426312,1,113


In [306]:
total_runs_df=total_runs_df[total_runs_df['inning']==1]

In [307]:
matches_df=matches.merge(total_runs_df[['match_id','total_runs']],left_on='id',right_on='match_id')

In [308]:
matches_df.columns

Index(['id', 'season', 'city', 'date', 'match_type', 'player_of_match',
       'venue', 'team1', 'team2', 'toss_winner', 'toss_decision', 'winner',
       'result', 'result_margin', 'target_runs', 'target_overs', 'super_over',
       'method', 'umpire1', 'umpire2', 'match_id', 'total_runs'],
      dtype='object')

In [309]:
matches_df=matches_df[['match_id','city','winner','total_runs']]

In [310]:
delivery_df=matches_df.merge(deliveries,on='match_id')

In [311]:
delivery_df=delivery_df[delivery_df['inning']==2]

In [312]:
delivery_df

,match_id,city,winner,total_runs_x,inning,batting_team,bowling_team,over,ball,batter,bowler,non_striker,batsman_runs,extra_runs,total_runs_y,extras_type,is_wicket,player_dismissed,dismissal_kind,fielder
124,335982,Bangalore,Kolkata Knight Riders,222,2,Royal Challengers Bangalore,Kolkata Knight Riders,0,1,R Dravid,AB Dinda,W Jaffer,1,0,1,NaN,0,NaN,NaN,NaN
125,335982,Bangalore,Kolkata Knight Riders,222,2,Royal Challengers Bangalore,Kolkata Knight Riders,0,2,W Jaffer,AB Dinda,R Dravid,0,1,1,wides,0,NaN,NaN,NaN
126,335982,Bangalore,Kolkata Knight Riders,222,2,Royal Challengers Bangalore,Kolkata Knight Riders,0,3,W Jaffer,AB Dinda,R Dravid,0,0,0,NaN,0,NaN,NaN,NaN
127,335982,Bangalore,Kolkata Knight Riders,222,2,Royal Challengers Bangalore,Kolkata Knight Riders,0,4,W Jaffer,AB Dinda,R Dravid,1,0,1,NaN,0,NaN,NaN,NaN
128,335982,Bangalore,Kolkata Knight Riders,222,2,Royal Challengers Bangalore,Kolkata Knight Riders,0,5,R Dravid,AB Dinda,W Jaffer,1,0,1,NaN,0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
240042,1426312,Chennai,Kolkata Knight Riders,113,2,Kolkata Knight Riders,Sunrisers Hyderabad,9,5,SS Iyer,AK Markram,VR Iyer,1,0,1,NaN,0,NaN,NaN,NaN
240043,1426312,Chennai,Kolkata Knight Riders,113,2,Kolkata Knight Riders,Sunrisers Hyderabad,9,6,VR Iyer,AK Markram,SS Iyer,1,0,1,NaN,0,NaN,NaN,NaN
240044,1426312,Chennai,Kolkata Knight Riders,113,2,Kolkata Knight Riders,Sunrisers Hyderabad,10,1,VR Iyer,Shahbaz Ahmed,SS Iyer,1,0,1,NaN,0,NaN,NaN,NaN
240045,1426312,Chennai,Kolkata Knight Riders,113,2,Kolkata Knight Riders,Sunrisers Hyderabad,10,2,SS Iyer,Shahbaz Ahmed,VR Iyer,1,0,1,NaN,0,NaN,NaN,NaN


In [313]:
delivery_df['current_score']=delivery_df.groupby('match_id')['total_runs_y'].cumsum()

In [314]:
delivery_df['runs_left']=delivery_df['total_runs_x']-delivery_df['current_score']

In [315]:
delivery_df

,match_id,city,winner,total_runs_x,inning,batting_team,bowling_team,over,ball,batter,...,batsman_runs,extra_runs,total_runs_y,extras_type,is_wicket,player_dismissed,dismissal_kind,fielder,current_score,runs_left
124,335982,Bangalore,Kolkata Knight Riders,222,2,Royal Challengers Bangalore,Kolkata Knight Riders,0,1,R Dravid,...,1,0,1,NaN,0,NaN,NaN,NaN,1,221
125,335982,Bangalore,Kolkata Knight Riders,222,2,Royal Challengers Bangalore,Kolkata Knight Riders,0,2,W Jaffer,...,0,1,1,wides,0,NaN,NaN,NaN,2,220
126,335982,Bangalore,Kolkata Knight Riders,222,2,Royal Challengers Bangalore,Kolkata Knight Riders,0,3,W Jaffer,...,0,0,0,NaN,0,NaN,NaN,NaN,2,220
127,335982,Bangalore,Kolkata Knight Riders,222,2,Royal Challengers Bangalore,Kolkata Knight Riders,0,4,W Jaffer,...,1,0,1,NaN,0,NaN,NaN,NaN,3,219
128,335982,Bangalore,Kolkata Knight Riders,222,2,Royal Challengers Bangalore,Kolkata Knight Riders,0,5,R Dravid,...,1,0,1,NaN,0,NaN,NaN,NaN,4,218
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
240042,1426312,Chennai,Kolkata Knight Riders,113,2,Kolkata Knight Riders,Sunrisers Hyderabad,9,5,SS Iyer,...,1,0,1,NaN,0,NaN,NaN,NaN,110,3
240043,1426312,Chennai,Kolkata Knight Riders,113,2,Kolkata Knight Riders,Sunrisers Hyderabad,9,6,VR Iyer,...,1,0,1,NaN,0,NaN,NaN,NaN,111,2
240044,1426312,Chennai,Kolkata Knight Riders,113,2,Kolkata Knight Riders,Sunrisers Hyderabad,10,1,VR Iyer,...,1,0,1,NaN,0,NaN,NaN,NaN,112,1
240045,1426312,Chennai,Kolkata Knight Riders,113,2,Kolkata Knight Riders,Sunrisers Hyderabad,10,2,SS Iyer,...,1,0,1,NaN,0,NaN,NaN,NaN,113,0


- Ball left

In [318]:
delivery_df['ball_left'] = 120 - (delivery_df['over']*6 + delivery_df['ball'])

In [319]:
delivery_df

,match_id,city,winner,total_runs_x,inning,batting_team,bowling_team,over,ball,batter,...,extra_runs,total_runs_y,extras_type,is_wicket,player_dismissed,dismissal_kind,fielder,current_score,runs_left,ball_left
124,335982,Bangalore,Kolkata Knight Riders,222,2,Royal Challengers Bangalore,Kolkata Knight Riders,0,1,R Dravid,...,0,1,NaN,0,NaN,NaN,NaN,1,221,119
125,335982,Bangalore,Kolkata Knight Riders,222,2,Royal Challengers Bangalore,Kolkata Knight Riders,0,2,W Jaffer,...,1,1,wides,0,NaN,NaN,NaN,2,220,118
126,335982,Bangalore,Kolkata Knight Riders,222,2,Royal Challengers Bangalore,Kolkata Knight Riders,0,3,W Jaffer,...,0,0,NaN,0,NaN,NaN,NaN,2,220,117
127,335982,Bangalore,Kolkata Knight Riders,222,2,Royal Challengers Bangalore,Kolkata Knight Riders,0,4,W Jaffer,...,0,1,NaN,0,NaN,NaN,NaN,3,219,116
128,335982,Bangalore,Kolkata Knight Riders,222,2,Royal Challengers Bangalore,Kolkata Knight Riders,0,5,R Dravid,...,0,1,NaN,0,NaN,NaN,NaN,4,218,115
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
240042,1426312,Chennai,Kolkata Knight Riders,113,2,Kolkata Knight Riders,Sunrisers Hyderabad,9,5,SS Iyer,...,0,1,NaN,0,NaN,NaN,NaN,110,3,61
240043,1426312,Chennai,Kolkata Knight Riders,113,2,Kolkata Knight Riders,Sunrisers Hyderabad,9,6,VR Iyer,...,0,1,NaN,0,NaN,NaN,NaN,111,2,60
240044,1426312,Chennai,Kolkata Knight Riders,113,2,Kolkata Knight Riders,Sunrisers Hyderabad,10,1,VR Iyer,...,0,1,NaN,0,NaN,NaN,NaN,112,1,59
240045,1426312,Chennai,Kolkata Knight Riders,113,2,Kolkata Knight Riders,Sunrisers Hyderabad,10,2,SS Iyer,...,0,1,NaN,0,NaN,NaN,NaN,113,0,58


- wicket left

In [322]:
wicket=delivery_df.groupby('match_id')['is_wicket'].cumsum()
delivery_df['wicket_left']=10-wicket

In [323]:
delivery_df.head(20)

,match_id,city,winner,total_runs_x,inning,batting_team,bowling_team,over,ball,batter,...,total_runs_y,extras_type,is_wicket,player_dismissed,dismissal_kind,fielder,current_score,runs_left,ball_left,wicket_left
124,335982,Bangalore,Kolkata Knight Riders,222,2,Royal Challengers Bangalore,Kolkata Knight Riders,0,1,R Dravid,...,1,NaN,0,NaN,NaN,NaN,1,221,119,10
125,335982,Bangalore,Kolkata Knight Riders,222,2,Royal Challengers Bangalore,Kolkata Knight Riders,0,2,W Jaffer,...,1,wides,0,NaN,NaN,NaN,2,220,118,10
126,335982,Bangalore,Kolkata Knight Riders,222,2,Royal Challengers Bangalore,Kolkata Knight Riders,0,3,W Jaffer,...,0,NaN,0,NaN,NaN,NaN,2,220,117,10
127,335982,Bangalore,Kolkata Knight Riders,222,2,Royal Challengers Bangalore,Kolkata Knight Riders,0,4,W Jaffer,...,1,NaN,0,NaN,NaN,NaN,3,219,116,10
128,335982,Bangalore,Kolkata Knight Riders,222,2,Royal Challengers Bangalore,Kolkata Knight Riders,0,5,R Dravid,...,1,NaN,0,NaN,NaN,NaN,4,218,115,10
129,335982,Bangalore,Kolkata Knight Riders,222,2,Royal Challengers Bangalore,Kolkata Knight Riders,0,6,W Jaffer,...,0,NaN,0,NaN,NaN,NaN,4,218,114,10
130,335982,Bangalore,Kolkata Knight Riders,222,2,Royal Challengers Bangalore,Kolkata Knight Riders,0,7,W Jaffer,...,0,NaN,0,NaN,NaN,NaN,4,218,113,10
131,335982,Bangalore,Kolkata Knight Riders,222,2,Royal Challengers Bangalore,Kolkata Knight Riders,1,1,R Dravid,...,0,NaN,1,R Dravid,bowled,NaN,4,218,113,9
132,335982,Bangalore,Kolkata Knight Riders,222,2,Royal Challengers Bangalore,Kolkata Knight Riders,1,2,V Kohli,...,0,NaN,0,NaN,NaN,NaN,4,218,112,9
133,335982,Bangalore,Kolkata Knight Riders,222,2,Royal Challengers Bangalore,Kolkata Knight Riders,1,3,V Kohli,...,4,legbyes,0,NaN,NaN,NaN,8,214,111,9


- current_runrate

In [325]:
delivery_df['crr']=round(delivery_df['current_score']*6/(120-delivery_df['ball_left']),2)

In [327]:
delivery_df

,match_id,city,winner,total_runs_x,inning,batting_team,bowling_team,over,ball,batter,...,extras_type,is_wicket,player_dismissed,dismissal_kind,fielder,current_score,runs_left,ball_left,wicket_left,crr
124,335982,Bangalore,Kolkata Knight Riders,222,2,Royal Challengers Bangalore,Kolkata Knight Riders,0,1,R Dravid,...,NaN,0,NaN,NaN,NaN,1,221,119,10,6.00
125,335982,Bangalore,Kolkata Knight Riders,222,2,Royal Challengers Bangalore,Kolkata Knight Riders,0,2,W Jaffer,...,wides,0,NaN,NaN,NaN,2,220,118,10,6.00
126,335982,Bangalore,Kolkata Knight Riders,222,2,Royal Challengers Bangalore,Kolkata Knight Riders,0,3,W Jaffer,...,NaN,0,NaN,NaN,NaN,2,220,117,10,4.00
127,335982,Bangalore,Kolkata Knight Riders,222,2,Royal Challengers Bangalore,Kolkata Knight Riders,0,4,W Jaffer,...,NaN,0,NaN,NaN,NaN,3,219,116,10,4.50
128,335982,Bangalore,Kolkata Knight Riders,222,2,Royal Challengers Bangalore,Kolkata Knight Riders,0,5,R Dravid,...,NaN,0,NaN,NaN,NaN,4,218,115,10,4.80
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
240042,1426312,Chennai,Kolkata Knight Riders,113,2,Kolkata Knight Riders,Sunrisers Hyderabad,9,5,SS Iyer,...,NaN,0,NaN,NaN,NaN,110,3,61,8,11.19
240043,1426312,Chennai,Kolkata Knight Riders,113,2,Kolkata Knight Riders,Sunrisers Hyderabad,9,6,VR Iyer,...,NaN,0,NaN,NaN,NaN,111,2,60,8,11.10
240044,1426312,Chennai,Kolkata Knight Riders,113,2,Kolkata Knight Riders,Sunrisers Hyderabad,10,1,VR Iyer,...,NaN,0,NaN,NaN,NaN,112,1,59,8,11.02
240045,1426312,Chennai,Kolkata Knight Riders,113,2,Kolkata Knight Riders,Sunrisers Hyderabad,10,2,SS Iyer,...,NaN,0,NaN,NaN,NaN,113,0,58,8,10.94


- rrr

In [330]:
delivery_df['rrr']=round((delivery_df['runs_left']*6)/delivery_df['ball_left'],2)

In [332]:
delivery_df

,match_id,city,winner,total_runs_x,inning,batting_team,bowling_team,over,ball,batter,...,is_wicket,player_dismissed,dismissal_kind,fielder,current_score,runs_left,ball_left,wicket_left,crr,rrr
124,335982,Bangalore,Kolkata Knight Riders,222,2,Royal Challengers Bangalore,Kolkata Knight Riders,0,1,R Dravid,...,0,NaN,NaN,NaN,1,221,119,10,6.00,11.14
125,335982,Bangalore,Kolkata Knight Riders,222,2,Royal Challengers Bangalore,Kolkata Knight Riders,0,2,W Jaffer,...,0,NaN,NaN,NaN,2,220,118,10,6.00,11.19
126,335982,Bangalore,Kolkata Knight Riders,222,2,Royal Challengers Bangalore,Kolkata Knight Riders,0,3,W Jaffer,...,0,NaN,NaN,NaN,2,220,117,10,4.00,11.28
127,335982,Bangalore,Kolkata Knight Riders,222,2,Royal Challengers Bangalore,Kolkata Knight Riders,0,4,W Jaffer,...,0,NaN,NaN,NaN,3,219,116,10,4.50,11.33
128,335982,Bangalore,Kolkata Knight Riders,222,2,Royal Challengers Bangalore,Kolkata Knight Riders,0,5,R Dravid,...,0,NaN,NaN,NaN,4,218,115,10,4.80,11.37
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
240042,1426312,Chennai,Kolkata Knight Riders,113,2,Kolkata Knight Riders,Sunrisers Hyderabad,9,5,SS Iyer,...,0,NaN,NaN,NaN,110,3,61,8,11.19,0.30
240043,1426312,Chennai,Kolkata Knight Riders,113,2,Kolkata Knight Riders,Sunrisers Hyderabad,9,6,VR Iyer,...,0,NaN,NaN,NaN,111,2,60,8,11.10,0.20
240044,1426312,Chennai,Kolkata Knight Riders,113,2,Kolkata Knight Riders,Sunrisers Hyderabad,10,1,VR Iyer,...,0,NaN,NaN,NaN,112,1,59,8,11.02,0.10
240045,1426312,Chennai,Kolkata Knight Riders,113,2,Kolkata Knight Riders,Sunrisers Hyderabad,10,2,SS Iyer,...,0,NaN,NaN,NaN,113,0,58,8,10.94,0.00


In [333]:
def result(row):
    return 1 if row['batting_team']==row['winner'] else 0

In [334]:
delivery_df['result']=delivery_df.apply(result,axis=1)

In [335]:
delivery_df

,match_id,city,winner,total_runs_x,inning,batting_team,bowling_team,over,ball,batter,...,player_dismissed,dismissal_kind,fielder,current_score,runs_left,ball_left,wicket_left,crr,rrr,result
124,335982,Bangalore,Kolkata Knight Riders,222,2,Royal Challengers Bangalore,Kolkata Knight Riders,0,1,R Dravid,...,NaN,NaN,NaN,1,221,119,10,6.00,11.14,0
125,335982,Bangalore,Kolkata Knight Riders,222,2,Royal Challengers Bangalore,Kolkata Knight Riders,0,2,W Jaffer,...,NaN,NaN,NaN,2,220,118,10,6.00,11.19,0
126,335982,Bangalore,Kolkata Knight Riders,222,2,Royal Challengers Bangalore,Kolkata Knight Riders,0,3,W Jaffer,...,NaN,NaN,NaN,2,220,117,10,4.00,11.28,0
127,335982,Bangalore,Kolkata Knight Riders,222,2,Royal Challengers Bangalore,Kolkata Knight Riders,0,4,W Jaffer,...,NaN,NaN,NaN,3,219,116,10,4.50,11.33,0
128,335982,Bangalore,Kolkata Knight Riders,222,2,Royal Challengers Bangalore,Kolkata Knight Riders,0,5,R Dravid,...,NaN,NaN,NaN,4,218,115,10,4.80,11.37,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
240042,1426312,Chennai,Kolkata Knight Riders,113,2,Kolkata Knight Riders,Sunrisers Hyderabad,9,5,SS Iyer,...,NaN,NaN,NaN,110,3,61,8,11.19,0.30,1
240043,1426312,Chennai,Kolkata Knight Riders,113,2,Kolkata Knight Riders,Sunrisers Hyderabad,9,6,VR Iyer,...,NaN,NaN,NaN,111,2,60,8,11.10,0.20,1
240044,1426312,Chennai,Kolkata Knight Riders,113,2,Kolkata Knight Riders,Sunrisers Hyderabad,10,1,VR Iyer,...,NaN,NaN,NaN,112,1,59,8,11.02,0.10,1
240045,1426312,Chennai,Kolkata Knight Riders,113,2,Kolkata Knight Riders,Sunrisers Hyderabad,10,2,SS Iyer,...,NaN,NaN,NaN,113,0,58,8,10.94,0.00,1


In [336]:
final_df=delivery_df[['batting_team','bowling_team','city','runs_left','ball_left','wicket_left','crr','rrr','result']]

In [337]:
final_df

,batting_team,bowling_team,city,runs_left,ball_left,wicket_left,crr,rrr,result
124,Royal Challengers Bangalore,Kolkata Knight Riders,Bangalore,221,119,10,6.00,11.14,0
125,Royal Challengers Bangalore,Kolkata Knight Riders,Bangalore,220,118,10,6.00,11.19,0
126,Royal Challengers Bangalore,Kolkata Knight Riders,Bangalore,220,117,10,4.00,11.28,0
127,Royal Challengers Bangalore,Kolkata Knight Riders,Bangalore,219,116,10,4.50,11.33,0
128,Royal Challengers Bangalore,Kolkata Knight Riders,Bangalore,218,115,10,4.80,11.37,0
...,...,...,...,...,...,...,...,...,...
240042,Kolkata Knight Riders,Sunrisers Hyderabad,Chennai,3,61,8,11.19,0.30,1
240043,Kolkata Knight Riders,Sunrisers Hyderabad,Chennai,2,60,8,11.10,0.20,1
240044,Kolkata Knight Riders,Sunrisers Hyderabad,Chennai,1,59,8,11.02,0.10,1
240045,Kolkata Knight Riders,Sunrisers Hyderabad,Chennai,0,58,8,10.94,0.00,1


to overcome biased performance we will shuffle the data 

In [356]:
final_df=final_df.sample(final_df.shape[0])

In [357]:
final_df

,batting_team,bowling_team,city,runs_left,ball_left,wicket_left,crr,rrr,result
168606,Sunrisers Hyderabad,Punjab Kings,NaN,92,96,10,8.50,5.75,0
37201,Sunrisers Hyderabad,Chennai Super Kings,Nagpur,136,115,10,2.40,7.10,0
132060,Royal Challengers Bangalore,Rajasthan Royals,Bengaluru,164,89,9,10.26,11.06,0
190293,Royal Challengers Bangalore,Rajasthan Royals,Mumbai,83,47,6,7.07,10.60,1
185442,Punjab Kings,Chennai Super Kings,Dubai,9,47,7,10.27,1.15,1
...,...,...,...,...,...,...,...,...,...
88730,Kolkata Knight Riders,Punjab Kings,Cuttack,35,35,9,8.05,6.00,1
8605,Sunrisers Hyderabad,Delhi Capitals,Delhi,174,108,10,10.00,9.67,0
73272,Royal Challengers Bangalore,Mumbai Indians,Mumbai,193,118,10,3.00,9.81,0
128916,Kolkata Knight Riders,Mumbai Indians,Kolkata,71,56,5,9.56,7.61,0


In [358]:
final_df.dropna(inplace=True)

In [371]:
final_df=final_df[final_df['ball_left']!=0]

In [373]:
X = final_df.iloc[:,:-1]
Y = final_df.iloc[:,-1]
x_train,x_test,y_train,y_test=train_test_split(X,Y,test_size=0.2,random_state=1)

In [375]:
x_train

,batting_team,bowling_team,city,runs_left,ball_left,wicket_left,crr,rrr
139003,Punjab Kings,Rajasthan Royals,Jaipur,134,92,7,5.14,8.74
149365,Chennai Super Kings,Kolkata Knight Riders,Chennai,74,93,9,7.56,4.77
121407,Kolkata Knight Riders,Punjab Kings,Kolkata,21,35,8,10.52,3.60
113537,Gujarat Titans,Punjab Kings,Rajkot,119,86,8,6.18,8.30
33076,Sunrisers Hyderabad,Mumbai Indians,Mumbai,140,101,9,10.11,8.32
...,...,...,...,...,...,...,...,...
87091,Royal Challengers Bangalore,Mumbai Indians,Mumbai,75,49,8,9.46,9.18
194150,Gujarat Titans,Chennai Super Kings,Pune,133,85,7,6.17,9.39
4433,Kolkata Knight Riders,Rajasthan Royals,Jaipur,142,79,7,7.90,10.78
222481,Lucknow Super Giants,Mumbai Indians,Chennai,82,25,1,6.32,19.68


In [377]:
trf=ColumnTransformer(
    [(
        'trf',OneHotEncoder(sparse_output=False, drop='first'),['batting_team','bowling_team','city']
    )],remainder='passthrough'
)

In [409]:
pipe=Pipeline(steps=[
    ('step1',trf),
    ('step2',LogisticRegression(solver='liblinear'))
])

In [411]:
pipe.fit(x_train,y_train)

Pipeline(steps=[('step1',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('trf',
                                                  OneHotEncoder(drop='first',
                                                                sparse_output=False),
                                                  ['batting_team',
                                                   'bowling_team', 'city'])])),
                ('step2', LogisticRegression(solver='liblinear'))])

In [413]:
y_pred=pipe.predict(x_test)

In [415]:
accuracy_score(y_test,y_pred)

0.7985368084133516

- we can try random forest, svm

In [420]:
pipe.predict_proba(x_test)[2]

array([0.64814788, 0.35185212])